In [ ]:
import ee

# Inicializar Earth Engine (ya autenticado)
ee.Initialize(project='ee-diegovaldesf')

# AOI: Pudahuel aeropuerto / industrial
aoi = ee.Geometry.Rectangle([-70.78, -33.46, -70.60, -33.33])

def mask_clouds_s2(image):
    qa = image.select('QA60')
    cloud_mask = qa.bitwiseAnd(1 << 10).eq(0).And(
        qa.bitwiseAnd(1 << 11).eq(0)
    )
    return image.updateMask(cloud_mask)

years = [2017, 2019, 2021, 2024]  # ajustado para Sentinel-2 estable

for year in years:
    start = f'{year}-01-01'
    end = f'{year}-02-28'

    collection = (
        ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
        .filterBounds(aoi)
        .filterDate(start, end)
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))
        .map(mask_clouds_s2)
    )

    n = collection.size().getInfo()
    print(f'{year}: {n} imágenes')

    if n == 0:
        print(f'⚠ Sin imágenes para {year}')
        continue

    composite = collection.median().clip(aoi)

    task = ee.batch.Export.image.toDrive(
        image=composite.select(['B2','B3','B4','B8','B11','B12']),
        description=f'sentinel2_pudahuel_{year}',
        folder='cambio_urbano_pudahuel',
        region=aoi, 
        scale=10,
        maxPixels=1e9
    )
    task.start()
    print(f'✔ Exportando {year}')


2017: 1 imágenes
✔ Exportando 2017
2019: 16 imágenes
✔ Exportando 2019
2021: 14 imágenes
✔ Exportando 2021
2024: 20 imágenes
✔ Exportando 2024
